In [2]:
%pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------

In [1]:
import pandas as pd
import re

# 1. Cấu hình tên file
input_file = 'Tu_encoded.csv' # Tên file csv của bạn
output_file = 'Huyen_encoded.csv'

try:
    # 1. Đọc dữ liệu từ file CSV
    # header=2 để khớp với cấu trúc file có 2 dòng tiêu đề thừa của bạn
    df = pd.read_csv(input_file, header=0)
    df.columns = df.columns.str.strip()
    
    print("Các cột tìm thấy trong file:", df.columns.tolist())
    print(f"Số lượng máy ban đầu: {len(df)}")

    # # 2. Drop các hàng mà cả 'title' và 'ram_type' đều trống (NaN)
    # # how='all' nghĩa là chỉ xóa khi cả 2 cột đều trống
    # df = df.dropna(subset=['title', 'ram_type'], how='all')
    
    # # Loại bỏ các dòng tiêu đề bị lặp lại (nếu có)
    # df = df[df['title'] != 'title']

    # 3. Hàm xử lý Storage (1TB -> 1024GB)
    def clean_storage(val):
        if pd.isna(val): return None
        s = str(val).upper().replace(' ', '')
        match = re.search(r'(\d+)', s) 
        if not match: return None
        num = int(match.group(1))
        if 'TB' in s:
            return num * 1024
        return num

    df['storage_size'] = df['storage_size'].apply(clean_storage)

    # 4. Preprocess RAM_TYPE (Cách tiếp cận linh hoạt)
    if 'ram_type' in df.columns:
        # Bước A: Chuyển về chuỗi, viết HOA, xóa khoảng trắng
        # Chúng ta tạm thời giữ NaN để không biến nó thành chữ "NAN"
        df['ram_type'] = df['ram_type'].apply(lambda x: str(x).strip().upper() if pd.notna(x) else x)
        
        # Bước B: Chỉ sửa những lỗi đánh máy cực kỳ rõ ràng
        # Các lỗi như DDR5x -> DDR5X đã tự động được giải quyết nhờ bước viết HOA ở trên
        df['ram_type'] = df['ram_type'].replace({
            'DDR45': 'DDR5'
            # '4.1' sẽ được giữ nguyên là '4.1' vì theo ý bạn nó có thể là chuẩn mới
        })

    # 5. Thực hiện One-Hot Encoding
    # dummy_na=False (mặc định): Các hàng có ram_type là NaN sẽ nhận giá trị 0 ở tất cả các cột ram mới
    # dtype=int: Để xuất ra số 0 và 1 thay vì True/False
    df = pd.get_dummies(df, columns=['ram_type'], prefix='ram_type', dtype=int, dummy_na=False)

    # 6. Xuất kết quả ra file CSV
    # encoding='utf-8-sig' để khi mở bằng Excel không bị lỗi font tiếng Việt
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    print(f"--- THÀNH CÔNG ---")
    print(f"Đã lưu file: {output_file}")
    print(f"Số lượng máy sau khi xử lý: {len(df)}")
    
    # Hiển thị các cột RAM mới để kiểm tra
    ram_cols = [col for col in df.columns if 'ram_type' in col]
    print("Các cột RAM sau khi Encoding:", ram_cols)

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

Các cột tìm thấy trong file: ['link', 'title', 'price_sale', 'price_base', 'brand', 'cpu_brand', 'cpu_detail', 'cpu_cores', 'cpu_threads', 'ram_size', 'ram_type', 'storage_size', 'storage_type', 'gpu_info', 'screen_size', 'screen_res', 'screen_tech', 'battery', 'raw_specs_count', 'price_sale_clean', 'gpu_brand', 'gpu_vram', 'gpu_class', 'gpu_brand_te', 'gpu_class_te', 'brand_score', 'cpu_gen', 'cpu_performance_score', 'cpu_brand_Intel', 'cpu_brand_AMD', 'cpu_brand_Apple']
Số lượng máy ban đầu: 6339
--- THÀNH CÔNG ---
Đã lưu file: Huyen_encoded.csv
Số lượng máy sau khi xử lý: 6339
Các cột RAM sau khi Encoding: ['ram_type_DDR3', 'ram_type_DDR3L', 'ram_type_DDR4', 'ram_type_DDR4X', 'ram_type_DDR4X4266MHZ', 'ram_type_DDR5', 'ram_type_DDR5X', 'ram_type_DDR6', 'ram_type_RAM_TYPE']
